在 LangChain 框架中,并行执行由 LangChain Expression Language(LCEL)实现。主要方法是将多个 runnable 组件结构化为字典或列表,当这些集合被传递给链中的下一个组件时,LCEL 运行时会并发执行其中的 runnable。

在 LangGraph 中,这一原理体现在图的拓扑结构。通过设计图结构,使多个无直接依赖的节点可由同一节点并发启动,这些并行路径独立执行,结果在后续汇聚节点整合。

In [ ]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model="deepseek-chat",
    temperature=0,
)

# ‐‐‐ 定义独立链 ‐‐‐
# 三个链分别执行不同任务,可并行运行
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请简明扼要地总结以下主题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

question_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请针对以下主题生成三个有趣的问题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请从以下主题中提取 5-10 个关键词,用逗号分隔:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

# ‐‐‐ 构建并行 + 汇总链 ‐‐‐

# 1. 定义并行任务块,结果与原始 topic 一起传递到下一步
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": question_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # 传递原始 topic
    }
)

# 2. 定义最终汇总 prompt,整合并行结果
synthesis_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """根据以下信息:
            摘要:{summary}
            相关问题:{questions}
            关键词:{key_terms}
            请综合生成完整答案。""",
        ),
        ("user", "原始主题:{topic}"),
    ]
)

# 3. 构建完整链,将并行结果直接传递给汇总 prompt,再由 LLM 和输出解析器处理
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

# ‐‐‐ 运行链 ‐‐‐
async def run_parallel_example(topic: str) -> None:
    """
    异步调用并行处理链,输出综合结果。
    Args:
    topic: 传递给 LangChain 的主题输入
    """
    print(f"\n--- 并行 LangChain 示例,主题:'{topic}' ---")
    try:
        # `ainvoke` 的输入是单个 topic 字符串,
        # 会传递给 map_chain 中的每个 runnable
        response = await full_parallel_chain.ainvoke(topic)
        print("\n- 最终响应 -")
        print(response)
    except Exception as e:
        print(f"\n 链执行出错:{e}")

if __name__ == "__main__":

    test_topic = "太空探索的历史"

    await run_parallel_example(test_topic)


--- 并行 LangChain 示例,主题:'太空探索的历史' ---

- 最终响应 -
太空探索是人类科技与勇气的壮丽史诗，自20世纪中叶开启以来，经历了从梦想到现实、从国家主导到多元合作的演进。以下是基于历史脉络、关键突破与未来展望的综合阐述：

---

### **一、太空探索的历史里程碑**
1. **1957年**：苏联发射首颗人造卫星 **“斯普特尼克1号”**，标志着太空时代的开端，引发美苏太空竞赛。  
2. **1961年**：苏联宇航员 **尤里·加加林** 乘坐“东方1号”首次进入太空，实现人类航天壮举。  
3. **1969年**：美国 **“阿波罗11号”** 任务成功，宇航员尼尔·阿姆斯特朗与巴兹·奥尔德林首次登月，成为人类探索的巅峰时刻。  
4. **1970-80年代**：  
   - 空间站时代开启：苏联“礼炮号”、美国“天空实验室”为长期太空居住奠定基础。  
   - 深空探测兴起：旅行者探测器飞向太阳系边缘，火星、金星探测器传回珍贵数据。  
5. **1981年**：美国 **航天飞机** 首飞，实现可重复使用航天器，降低近地轨道任务成本。  
6. **1990年代至今**：  
   - **国际空间站（ISS）** 建成，成为多国合作的象征。  
   - 火星探测密集化：“好奇号”“毅力号”寻找生命痕迹，中国“天问一号”实现绕落巡。  
   - 商业航天崛起：SpaceX、蓝色起源等企业推动火箭回收、卫星互联网和太空旅游。

---

### **二、关键问题深度解析**
#### **1. 阿波罗计划前的技术突破与尝试**
- **火箭技术**：德国V-2火箭为基础，美苏改进研制出“土星五号”（美国）和“联盟号”（苏联）等重型运载火箭。  
- **载人航天准备**：苏联“东方计划”验证生命支持系统；美国“水星计划”“双子座计划”测试太空行走、轨道交会对接等关键技术。  
- **无人探测铺垫**：月球探测器（如苏联“月球号”）传回表面图像，确认着陆可行性。  
- **计算机与导航**：阿波罗导航计算机（AGC）首次实现实时轨道计算，虽内存仅2KB，但可靠性极高。

#### **2. 国际空间站的技术协调与合作分工**
- **标准统一**：以美国NASA和俄罗斯航天局为主导，协调舱段接口、通信

In [5]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model="deepseek-chat",
    temperature=0,
)

# ‐‐‐ 定义独立链 ‐‐‐
# 三个链分别执行不同任务,可并行运行
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请简明扼要地总结以下主题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

question_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请针对以下主题生成三个有趣的问题:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "请从以下主题中提取 5-10 个关键词,用逗号分隔:"),
            ("user", "{topic}"),
        ]
    )
    | llm
    | StrOutputParser()
)

# ‐‐‐ 构建并行 + 汇总链 ‐‐‐

# 1. 定义并行任务块,结果与原始 topic 一起传递到下一步
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": question_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # 传递原始 topic
    }
)

# 2. 定义最终汇总 prompt,整合并行结果
synthesis_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """根据以下信息:
            摘要:{summary}
            相关问题:{questions}
            关键词:{key_terms}
            请综合生成完整答案。""",
        ),
        ("user", "原始主题:{topic}"),
    ]
)

# 3. 构建完整链,将并行结果直接传递给汇总 prompt,再由 LLM 和输出解析器处理
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()

# ‐‐‐ 运行链 ‐‐‐
async def run_parallel_example(topic: str) -> None:
    """
    异步调用并行处理链,输出综合结果。
    Args:
    topic: 传递给 LangChain 的主题输入
    """
    print(f"\n--- 并行 LangChain 示例,主题:'{topic}' ---")
    try:
        # 步骤1: 执行并行任务，获取中间结果
        print("\n[步骤1] 开始并行执行三个任务...")
        parallel_results = await map_chain.ainvoke(topic)
        
        # 显示中间结果
        print("\n[中间结果] 并行任务执行完成:")
        print(f"\n  📝 摘要 (summary):")
        print(f"  {parallel_results['summary']}")
        print(f"\n  ❓ 问题 (questions):")
        print(f"  {parallel_results['questions']}")
        print(f"\n  🔑 关键词 (key_terms):")
        print(f"  {parallel_results['key_terms']}")
        print(f"\n  📌 原始主题 (topic):")
        print(f"  {parallel_results['topic']}")
        
        # 步骤2: 执行汇总步骤
        print("\n[步骤2] 开始汇总整合...")
        synthesis_result = await (synthesis_prompt | llm | StrOutputParser()).ainvoke(parallel_results)
        
        print("\n[最终响应]")
        print(synthesis_result)
        
    except Exception as e:
        print(f"\n❌ 链执行出错: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":

    test_topic = "太空探索的历史"

    await run_parallel_example(test_topic)


--- 并行 LangChain 示例,主题:'太空探索的历史' ---

[步骤1] 开始并行执行三个任务...

[中间结果] 并行任务执行完成:

  📝 摘要 (summary):
  太空探索始于20世纪中叶，主要里程碑包括：

1. **1957年**：苏联发射首颗人造卫星“斯普特尼克1号”，开启太空时代。
2. **1961年**：苏联宇航员加加林成为首位进入太空的人类。
3. **1969年**：美国“阿波罗11号”任务实现首次载人登月。
4. **1970年代**：空间站兴起（如礼炮号、天空实验室），深空探测器探索行星。
5. **1981年**：美国航天飞机首飞，实现可重复使用航天器。
6. **1998年**：国际空间站开始建设，开启多国长期合作。
7. **21世纪**：私营企业加入（如SpaceX），推动商业航天；探测器探索火星、外行星及太阳系边缘。
8. **未来目标**：重返月球、载人火星任务及深空探索。

太空探索推动了科技、通信和地球科学研究，成为人类探索未知的重要领域。

  ❓ 问题 (questions):
  1. 在阿波罗计划之前，人类为登月做了哪些关键的技术突破和尝试？  
2. 国际空间站的建设过程中，不同国家之间如何协调技术标准与合作分工？  
3. 未来商业太空旅行可能如何改变普通人对宇宙的认知和生活方式？

  🔑 关键词 (key_terms):
  太空探索,历史,航天,宇航员,火箭,卫星,登月,国际空间站,探测器,航天飞机

  📌 原始主题 (topic):
  太空探索的历史

[步骤2] 开始汇总整合...

[最终响应]
太空探索是人类探索未知、拓展生存边界的重要历程，其历史跨越了从20世纪中叶至今的多个关键阶段。以下是结合**太空探索的历史**主线，对相关技术与里程碑的梳理，并针对具体问题的延伸分析：

---

### **一、太空探索的历史主线**
1. **起步阶段（1950-1960年代）**  
   - **1957年**：苏联发射首颗人造卫星“斯普特尼克1号”，标志着太空时代的开启。  
   - **1961年**：苏联宇航员**加加林**乘坐“东方1号”首次进入太空，实现载人航天突破。  
   - **1969年**：美国“阿波罗11号”任务成功登月，**阿姆斯特朗**成为首